In [4]:
import pandas as pd
from pathlib import Path


DATA_DIR = (Path().resolve().parent / "data" )

df_raw = pd.read_csv(DATA_DIR / "02_processed" / "features_raw.csv")
df_raw.head()

/tmp/ipykernel_27881/344496254.py:7: DtypeWarning: Columns (60,61,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  df_raw = pd.read_csv(DATA_DIR / "02_processed" / "features_raw.csv")


,Median_age,M0_4,M5_9,M10_14,M15_19,M20_24,M25_29,M30_34,M35_39,M40_44,...,OBS_VALUE,Males,Females,Persons,Sex_ratio,propertyUnitNumber,propertyHouseNumber,settlementDate,natureOfProperty,primaryPurpose
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,20,59,20211223.0,R,RESIDENCE
1,41.5,636.0,710.0,774.0,696.0,618.0,408.0,522.0,610.0,722.0,...,16600.0,10048.0,10574.0,20622.0,95.0,NaN,28,20211223.0,R,RESIDENCE
2,43.4,349.0,430.0,457.0,414.0,303.0,234.0,284.0,344.0,403.0,...,9605.0,6038.0,5907.0,11945.0,102.2,NaN,808 C,20211223.0,V,VACANT LAND
3,33.4,429.0,453.0,376.0,500.0,1124.0,1037.0,720.0,571.0,459.0,...,14159.0,8607.0,8147.0,16754.0,105.6,NaN,51,20211223.0,R,RESIDENCE
4,38.6,338.0,333.0,331.0,310.0,284.0,219.0,272.0,308.0,301.0,...,7176.0,4606.0,4482.0,9088.0,102.8,NaN,16,20211224.0,R,RESIDENCE


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

# Load your dataset
df = pd.read_csv('your_dataset.csv')

# Preprocessing
# Convert OBS_VALUE to numeric
df['OBS_VALUE'] = pd.to_numeric(df['OBS_VALUE'], errors='coerce')

# Handle missing values here (e.g., with imputation or removal)

# Define which columns are categorical
categorical_features = ['natureOfProperty', 'primaryPurpose', 'propertyUnitNumber', 'propertyHouseNumber']
# Define which columns are numerical (excluding 'OBS_VALUE')
numerical_features = df.columns.difference(['OBS_VALUE'] + categorical_features).tolist()

# Define the preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Define the model
model = Lasso(alpha=0.1)

# Create the pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

# Split the data into training and test sets
X = df.drop('OBS_VALUE', axis=1)
y = df['OBS_VALUE'].dropna()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Fit the model
pipeline.fit(X_train, y_train)

# Predict and evaluate
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Get model coefficients for interpretation
coefs = pipeline.named_steps['model'].coef_
# Match coefficients to column names in preprocessed data
columns = (pipeline.named_steps['preprocessor'].transformers_[0][2] +
           list(pipeline.named_steps['preprocessor'].named_transformers_['cat'].get_feature_names()))
coef_df = pd.DataFrame(coefs, index=columns, columns=['Coefficients'])